# I. $LU$ factorization of a square matrix

Consider a simple naive implementation of the LU decomposition. 

Note that we're using the `numpy` arrays to represent matrices [do **not** use `np.matrix`].

In [38]:
import numpy as np

def diy_lu(a):
    """Construct the LU decomposition of the input matrix.
    
    Naive LU decomposition: work column by column, accumulate elementary triangular matrices.
    No pivoting.
    """
    N = a.shape[0]
    
    u = a.copy()
    L = np.eye(N)
    for j in range(N-1):
        lam = np.eye(N)
        gamma = u[j+1:, j] / u[j, j]
        lam[j+1:, j] = -gamma
        u = lam @ u

        lam[j+1:, j] = gamma
        L = L @ lam
    return L, u

In [39]:
# Now, generate a full rank matrix and test the naive implementation

import numpy as np

N = 6
a = np.zeros((N, N), dtype=float)
for i in range(N):
    for j in range(N):
        a[i, j] = 3. / (0.6*i*j + 1)

np.linalg.matrix_rank(a)

6

In [40]:
# Tweak the printing of floating-point numbers, for clarity
np.set_printoptions(precision=3)

In [41]:
L, u = diy_lu(a)

print(L, "\n")
print(u, "\n")

# Quick sanity check: L times U must equal the original matrix, up to floating-point errors.
print(L@u - a)

[[1.    0.    0.    0.    0.    0.   ]
 [1.    1.    0.    0.    0.    0.   ]
 [1.    1.455 1.    0.    0.    0.   ]
 [1.    1.714 1.742 1.    0.    0.   ]
 [1.    1.882 2.276 2.039 1.    0.   ]
 [1.    2.    2.671 2.944 2.354 1.   ]] 

[[ 3.000e+00  3.000e+00  3.000e+00  3.000e+00  3.000e+00  3.000e+00]
 [ 0.000e+00 -1.125e+00 -1.636e+00 -1.929e+00 -2.118e+00 -2.250e+00]
 [ 0.000e+00  0.000e+00  2.625e-01  4.574e-01  5.975e-01  7.013e-01]
 [ 0.000e+00  1.110e-16  0.000e+00 -2.197e-02 -4.480e-02 -6.469e-02]
 [ 0.000e+00 -2.819e-16  0.000e+00  0.000e+00  8.080e-04  1.902e-03]
 [ 0.000e+00  3.369e-16  0.000e+00 -1.541e-18  2.168e-19 -1.585e-05]] 

[[ 0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00]
 [ 0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00]
 [ 0.000e+00  0.000e+00  0.000e+00  2.220e-16 -1.110e-16 -1.665e-16]
 [ 0.000e+00  0.000e+00  2.220e-16 -5.551e-17 -1.665e-16 -1.665e-16]
 [ 0.000e+00  0.000e+00 -1.110e-16  2.776e-16 -2.776e-16  5.551e-17]
 

# II. The need for pivoting

Let's tweak the matrix a little bit, we only change a single element:

In [42]:
a1 = a.copy()
a1[1, 1] = 3

Resulting matix still has full rank, but the naive LU routine breaks down.

In [43]:
np.linalg.matrix_rank(a1)

6

In [44]:
l, u = diy_lu(a1)

print(l, u)

[[nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]] [[nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]]


//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:15: RuntimeWarning: divide by zero encountered in true_divide
  from ipykernel import kernelapp as app
//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:17: RuntimeWarning: invalid value encountered in matmul
//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:20: RuntimeWarning: invalid value encountered in matmul
//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:15: RuntimeWarning: invalid value encountered in true_divide
  from ipykernel import kernelapp as app


### Test II.1

For a naive LU decomposition to work, all leading minors of a matrix should be non-zero. Check if this requirement is satisfied for the two matrices `a` and `a1`.

(20% of the grade)

Тут просто обрезаем нашу матрицу и находим ее ведущие миноры, то есть определители этих обрезанных матриц

Если нулевых миноров нет, то выдаем True, если есть - False 

In [45]:
def f(a):
    length = a.shape[0]
    for i in range(0,length):
        if np.linalg.det(a[:i,:i]) == 0:
            return False
    return True   

### Test II.2

Modify the `diy_lu` routine to implement column pivoting. Keep track of pivots, you can either construct a permutation matrix, or a swap array (your choice).

(40% of the grade)

Implement a function to reconstruct the original matrix from a decompositon. Test your routines on the matrices `a` and `a1`.

(40% of the grade)

Посмотрим, будет ли работать наша функция для a и a1

In [46]:
f(a)

True

Все нормально, и Lu разложение существует, и оно найдено выше

In [47]:
f(a1)

False

Получили False, то есть один из ведущих миноров нулевой, как и получилось выше

Наша функция f(a) для выявления нулевых ведущих миноров работает

Создадим теперь вспомогающие функции для составления матрицы перестановок, самой перестановок

In [48]:
def change(A,j,i):
    A1 = A.copy()
    b = np.zeros_like(A[i])
    b = A1[i].copy()
    A1[i] = A1[j]
    A1[j] = b
    return A1
#change - матрица А, в которой i и j строки поменяны местами

In [49]:
def change1(a,i,j):
    b = np.eye((a.shape[0]),dtype=float)
    b[i][i] = 0
    b[j][j] = 0
    b[i][j] = 1
    b[j][i] = 1
    return b
#change1 - матрица перестановок, чтобы поменять строки надо change1 умножить на a 

Матрица change1 создана для того, чтобы запомнить, какие мы перестановки совершаем

Так как я не умею выделять из многомерного массива в конкретном срезе максимальный элемент, просто напишу функцию, которая будет это делать вместо меня

In [50]:
def index_max(a):
    b = []
    b = [a[i] for i in range(0,len(a))]
    return b.index(max(b))

Эта функция помогает мне определять индекс максималного элемента у срезанного массива

In [51]:
def det(A):
    length = A.shape[0] #создаем размер нашей матрицы
    det1 = np.zeros((length,length)) #создаем многомерный массив, который будет помогать нам определять индекс строки
    #при которой детерминант нулевого ведущего минора будет максимальным
    permutation = np.eye(length) #создаем изначально единичную матрицу, которая потом будет превращаться
    #в матрицу перестановок
    for i in range(0,length):
        if np.linalg.det(A[:i,:i]) == 0: #находим нулевой ведущий минор
            for j in range(i,length):
                det1[i][j] = (np.absolute(np.linalg.det(change(A,i-1,j)[:i,:i]))) #после того, как нашли
                #наполняем наш массив значениями ведущих миноров, которые получаются при замене строк
            index = index_max(det1[i])#после чего находим номер строчки, при которой уже ненулевой минор 
            #будет максимальным по модулю
            permutation = change1(A,i-1,index) @ permutation #после этого находим нашу матрицу перестановок
            #чтобы поменять нашу матрицу a1
    return permutation


После этого находим нашу преобразованную матрицу, которая уже может быть представлена в Lu виде

In [52]:
def PA(A):
    return det(A) @ A

Сейчас мы имеем равенство 
$Pa_{1} = Lu $

Тогда имеем: $a_{1} = P^{-1}Lu$

In [61]:
def P1(a): 
    return np.linalg.inv(a)

На этом написание функций закончено, и можно приступить к проверке работы этого всего

In [62]:
det(a1)

array([[1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1.],
       [0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 1., 0.],
       [0., 1., 0., 0., 0., 0.]])

Получили матрицу перестановок

In [63]:
PA(a1)

array([[3.   , 3.   , 3.   , 3.   , 3.   , 3.   ],
       [3.   , 0.75 , 0.429, 0.3  , 0.231, 0.188],
       [3.   , 1.364, 0.882, 0.652, 0.517, 0.429],
       [3.   , 1.071, 0.652, 0.469, 0.366, 0.3  ],
       [3.   , 0.882, 0.517, 0.366, 0.283, 0.231],
       [3.   , 3.   , 1.364, 1.071, 0.882, 0.75 ]])

Так выглядит наша преобразованная матрица

In [64]:
P1(det(a1))

array([[ 1.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  1.],
       [ 0.,  0.,  1.,  0.,  0.,  0.],
       [-0., -0., -0.,  1., -0., -0.],
       [ 0.,  0.,  0.,  0.,  1.,  0.],
       [ 0.,  1.,  0.,  0.,  0.,  0.]])

Нашли обратную матрицу перестановок 

Находим Lu разложение для преобразованной матрицы

In [65]:
L, u = diy_lu(PA(a1))

In [66]:
diy_lu(PA(a1))

(array([[1.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00],
        [1.000e+00, 1.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00],
        [1.000e+00, 7.273e-01, 1.000e+00, 0.000e+00, 0.000e+00, 0.000e+00],
        [1.000e+00, 8.571e-01, 5.807e-01, 1.000e+00, 0.000e+00, 0.000e+00],
        [1.000e+00, 9.412e-01, 2.529e-01, 6.797e-01, 1.000e+00, 0.000e+00],
        [1.000e+00, 0.000e+00, 6.611e+00, 9.937e+01, 2.597e+03, 1.000e+00]]),
 array([[ 3.000e+00,  3.000e+00,  3.000e+00,  3.000e+00,  3.000e+00,
          3.000e+00],
        [ 0.000e+00, -2.250e+00, -2.571e+00, -2.700e+00, -2.769e+00,
         -2.812e+00],
        [ 0.000e+00,  0.000e+00, -2.475e-01, -3.842e-01, -4.688e-01,
         -5.260e-01],
        [ 0.000e+00,  1.110e-16,  0.000e+00,  6.152e-03,  1.172e-02,
          1.617e-02],
        [ 0.000e+00, -1.310e-16,  0.000e+00,  0.000e+00, -7.044e-05,
         -1.585e-04],
        [ 0.000e+00,  3.291e-13,  0.000e+00,  2.282e-17,  0.000e+00,
          3.202e-0

И тогда наша искомая матрица будет равна $a_{1} = P^{-1}Lu$

In [67]:
P1(det(a1)) @ L @ u

array([[3.   , 3.   , 3.   , 3.   , 3.   , 3.   ],
       [3.   , 3.   , 1.364, 1.071, 0.882, 0.75 ],
       [3.   , 1.364, 0.882, 0.652, 0.517, 0.429],
       [3.   , 1.071, 0.652, 0.469, 0.366, 0.3  ],
       [3.   , 0.882, 0.517, 0.366, 0.283, 0.231],
       [3.   , 0.75 , 0.429, 0.3  , 0.231, 0.188]])

Сравним теперь

In [68]:
print(P1(det(a1))@L@u - a1)

[[ 0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00]
 [ 0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00  2.220e-16]
 [ 0.000e+00  0.000e+00  0.000e+00  2.220e-16 -1.110e-16 -1.665e-16]
 [ 0.000e+00  0.000e+00  2.220e-16 -5.551e-17 -1.665e-16  2.776e-16]
 [ 0.000e+00  0.000e+00 -1.110e-16 -1.665e-16  1.665e-16 -3.886e-16]
 [ 0.000e+00  0.000e+00 -1.665e-16 -1.665e-16  5.551e-17  0.000e+00]]


Точность неплохая получилась, и у нас все заработало!)